# **1-on_time.csv**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Data Import
1. On-time performance
2. Daily weather data (pending)
3. Traffic/construction data (pending)

In [ ]:
DF = pd.read_csv("../../datasets/otp_Oct26-Dec09.csv",low_memory=False)
for col in DF.columns:
    print(col,DF[col].dtype)
DF

# Data Preprocessing
1. Remove Row ID
2. Convert Scheduled Time column to DateTime format
3. Convert Location column to tuple
4. Group Route Name and Route Destination into a single column 
5. Add Delay Type (Early, On-time, Short, Medium, Long, Severe, Crippling)

In [ ]:
# 1
DF = DF.drop(columns="Row ID")
# 2
DF['Scheduled Time'] = pd.to_datetime(DF['Scheduled Time'])
print(DF['Scheduled Time'].dtype)
DF.head()
# 3
def __clean_string(df):
    to_repl = ''
    for pat in ["POINT (",")"]:
        df = df.str.replace(pat,to_repl,regex=False)
    df = df.str.split()
    return df

def as_dtype(lst,dtype):
    if isinstance(lst,(list,tuple)):
        return type(lst)([dtype(i) for i in lst])
    else: return lst

DF['Location'] = __clean_string(DF['Location'])
DF.loc[:,['Long','Lat']] = [as_dtype(i,float) for i in DF['Location'].values]
DF = DF.drop(columns="Location")
DF
# 4
DF['Route'] = tuple([tuple(val) for val in DF[['Route Number','Route Name','Route Destination']].values])
DF = DF.drop(columns=['Route Name','Route Destination'])
# DF.to_csv("clean_datasets/ON_TIME.csv")
DF


Delay types:
- Early: < -2 min
- On-time: Within -2 min - <2 min
- Short delay: 2 min - <10 min,
- Medium delay: 10 min - <30 min,
- Long delay: 30 min - <60 min,
- Severe delay: 60+ min

In [ ]:
DF.to_csv("clean_datasets/ON_TIME_NOV_2021.csv")

In [ ]:
DF = pd.read_csv("clean_datasets/NOV_2021/ON_TIME_NOV_2021.csv").drop(columns=["Unnamed: 0"])
DF

In [ ]:
START_DATE = pd.to_datetime('Nov 1 2021'); END_DATE = pd.to_datetime('Dec 1 2021')
cond = (START_DATE <= pd.to_datetime(DF["Scheduled Time"])) & (pd.to_datetime(DF["Scheduled Time"]) <= END_DATE)
DF = DF[cond].sort_values('Scheduled Time')
DF

In [ ]:
DF.to_csv("clean_datasets/NOV_2021/ON_TIME_NOV_2021.csv")

# **2-traffic_counts.csv**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DF = pd.read_csv("../../datasets/tc2019-Dec2021.csv",low_memory=False)
DF
for col in DF.columns:
    print(col,DF[col].dtype)
# Data Preprocessing
# 1. Convert Timestamp, selecting only periods of interest
# 2. Analyze relationship between Site x Location and {Right,Left} x {N,S,W,E}
# 3. Clean columns if possible
#1
DF.loc[:,'Timestamp'] = pd.to_datetime(DF['Timestamp'])
print(DF['Timestamp'].dtype)
START_DATE = pd.to_datetime('Nov 1 2021'); END_DATE = pd.to_datetime('Dec 1 2021')
cond = (START_DATE <= DF["Timestamp"]) & (DF["Timestamp"] <= END_DATE)
DF = DF[cond].sort_values('Timestamp')
sites = DF.loc[:,"Site"].str.split().values
DF.loc[:,["Street","Near"]] = [[l[0],l[-1]] for l in sites]
DF = DF.drop(columns=["Right","Left","Location"]).rename(columns={"Latitude":"Lat","Longitude":"Long"})
DF.to_csv("clean_datasets/TRAFFIC_COUNTS_NOV_2021.csv")
DF

In [ ]:
#2
DF_SITES = DF[["Site","Street","Near","Lat","Long"]].drop_duplicates().set_index("Site")
DF_SITES.to_csv("clean_datasets/SITES.csv")
DF_SITES

# **3-stops.csv**

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
DF = pd.read_csv("../../datasets/stops.csv",low_memory=False)
DF

In [ ]:
for col in DF.columns:
    print(col,DF[col].dtype)

In [ ]:
df = DF[DF["stop_name"].str.contains("Station")][["stop_id","stop_name"]]
df.head(25)

In [ ]:
[[l[0].split()[0]," ".join(l[0].split()[1:]),l[1]] for l in df["stop_name"].str.split(" at ")]

In [ ]:
DF.loc[:,["Direction","Street","At"]] = [[l[0].split()[0]," ".join(l[0].split()[1:]),l[1]] for l in DF["stop_name"].str.split(" at ")]
DF = DF.rename(columns={"stop_id":"Stop Number","stop_lat":"Lat","stop_lon":"Long","stop_name":"Stop Name"})
DF = DF.drop(columns=["stop_code","stop_url"])
DF

In [ ]:
DF.to_csv("clean_datasets/STOPS.csv")

# **4-closure.csv**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
DF = pd.read_csv("clean_datasets/NOV_2021/LANE_CLOSURE_NOV_1.csv",low_memory=False)
DF.head()

,Primary Street,Cross Street,Boundaries,Direction,Date Closed - From,Date Closed - To,Traffic Effect,Organization,Time Closed - From,Time Closed - To,...,Inserted Date,Geometry Id,Status,Complete Closure,KML,Latitude,Longitude,X,Y,Geometry
0,Forrester Av,St Annes Rd,St Annes Rd to Gascon Rd,Eastbound & Westbound,December 10 2021,December 15 2021,"Complete closure of all eastbound lanes, compl...",Rocky Road recycling,12:00 AM,12:00 AM,...,12/10/2021 08:21:19 AM,557721,Current,Yes,<LineString><extrude>0</extrude><tessellate>0<...,49.832268,-97.091898,637219.053852,5.521728e+06,MULTILINESTRING ((-97.093977146251 49.83148703...
1,Main St,York Av,York Av to Broadway,Southbound,December 16 2021,December 16 2021,"Southbound curb lane, southbound middle lane, ...",Skyline Crane,09:00 AM,03:00 PM,...,12/10/2021 03:37:55 PM,557755,Future,No,<LineString><extrude>0</extrude><tessellate>0<...,49.889631,-97.135425,633930.314298,5.528026e+06,MULTILINESTRING ((-97.135046852214 49.88889353...
2,Grandin St,Tache Av,Tache Av to St Joseph St,Eastbound,May 31 2021,January 13 2022,"South side sidewalk, closed for building const...",Qualico,12:00 AM,12:00 AM,...,05/26/2021 03:38:27 PM,550361,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.896013,-97.126315,634566.885434,5.528752e+06,MULTILINESTRING ((-97.127616387474 49.89567885...
3,Lagimodiere Bv,Cottonwood Rd,Cottonwood Rd to Paterson St,Southbound,December 13 2021,December 14 2021,"Southbound curb lane, closed for electrical/li...",Wawasum Energy,12:00 AM,12:00 AM,...,12/01/2021 01:38:09 PM,557730,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.861131,-97.057245,639627.700688,5.525001e+06,MULTILINESTRING ((-97.054241146686 49.85979844...
4,Niakwa Rd,Seagrim Rd,Seagrim Rd to End,Eastbound,June 28 2021,December 17 2021,"Westbound curb lane, closed for building const...",Gardon Construction,12:00 AM,12:00 AM,...,06/23/2021 01:24:18 PM,551194,Current,No,<LineString><extrude>0</extrude><tessellate>0<...,49.855435,-97.098817,636656.182968,5.524291e+06,MULTILINESTRING ((-97.100551636527 49.85485775...


In [ ]:
for col in DF.columns:
    print(col,DF[col].dtype)

In [13]:
DF = pd.read_csv("clean_datasets/NOV_2021/LANE_CLOSURE_NOV_1.csv",low_memory=False)
DF = DF.rename(columns={"Primary Street":"Street","Cross Street":"At","Latitude":"Lat","Longitude":"Long"})
DF.loc[:,"Street"] = [" ".join(l[:-1]) for l in DF["Street"].str.split()]
DF.loc[:,"At"] = [" ".join(l[:-1]) for l in DF["At"].str.split()]
DF.loc[:,"Boundaries"] = DF["Boundaries"].str.split(" to ")
DF.loc[:,"Direction"] = DF["Direction"].str.split(" & ")
__clean_string = lambda l: [[[(float(j.split()[1]),float(j.split()[0])) for j in s.split(", ")] for s in m] for m in l.values]
DF.loc[:,"Geometry"] = __clean_string(DF["Geometry"].str.replace("MULTILINESTRING \(\(|\)\)","",regex=True).replace().str.split("\), \("))
# DF["Geometry"] = DF["Geometry"].str.strip("MULTILINESTRING \(|\)").str.split("\), \(")
# DF["Geometry"] = [[to_tuple_l(s,rev=True,dtype=float,strip_l="(|)",split_l=", ",split=" ") for s in m] for m in DF["Geometry"].values]
DF["Date Closed - From"] = pd.to_datetime(DF["Date Closed - From"]) 
DF["Date Closed - To"] = pd.to_datetime(DF["Date Closed - To"]) 

def _num_points(m): 
    set_i = set()
    for l in m: set_i |= set(l)
    return set_i 
DF["Num_Segments"] = [len(i) for i in DF["Geometry"].values]
DF["Num_Points"] = [len(_num_points(i)) for i in DF["Geometry"].values]

DF = DF.drop(columns=["Traffic Effect","Organization","Lane Closure ID","Modified Date","Inserted Date","Geometry Id","Status","KML","X","Y"])
DF

/Users/jasetran/.local/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3162: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim


,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Forrester,St Annes,"[St Annes Rd, Gascon Rd]","[Eastbound, Westbound]",2021-12-10,2021-12-15,12:00 AM,12:00 AM,Yes,49.832268,-97.091898,"[[(49.831487032554, -97.093977146251), (49.831...",1,20
1,Main,York,"[York Av, Broadway]",[Southbound],2021-12-16,2021-12-16,09:00 AM,03:00 PM,No,49.889631,-97.135425,"[[(49.888893537771, -97.135046852214), (49.889...",1,5
2,Grandin,Tache,"[Tache Av, St Joseph St]",[Eastbound],2021-05-31,2022-01-13,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
3,Lagimodiere,Cottonwood,"[Cottonwood Rd, Paterson St]",[Southbound],2021-12-13,2021-12-14,12:00 AM,12:00 AM,No,49.861131,-97.057245,"[[(49.859798445742, -97.054241146686), (49.859...",1,12
4,Niakwa,Seagrim,"[Seagrim Rd, End]",[Eastbound],2021-06-28,2021-12-17,12:00 AM,12:00 AM,No,49.855435,-97.098817,"[[(49.854857752526, -97.100551636527), (49.854...",1,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Kenaston,South Town,"[South Town Rd, Bison Dr]",[Northbound],2021-08-12,2022-04-30,12:00 AM,12:00 AM,No,49.793520,-97.193865,"[[(49.79534709946, -97.194058236482), (49.7941...",1,56
155,Smith,York,"[York Av, St Mary Av]",[Northbound],2019-07-08,2021-12-17,09:00 AM,02:00 PM,No,49.890207,-97.140148,"[[(49.889472952638, -97.139749173077), (49.889...",1,4
156,Juno,McDermot,"[McDermot Av, Notre Dame Av]","[Northbound, Southbound]",2021-02-21,2021-12-30,12:00 AM,12:00 AM,No,49.900115,-97.152503,"[[(49.899456360592, -97.153010204315), (49.899...",1,4
157,Fort,Graham,"[Graham Av, Portage Av]",[Northbound],2021-08-17,2023-10-27,12:00 AM,12:00 AM,No,49.893978,-97.139238,"[[(49.893005589417, -97.138726184172), (49.893...",1,6


In [12]:
DF.to_csv("clean_datasets/LANE_CLOSURE_2.csv")

In [9]:
DF1 = pd.read_csv("clean_datasets/LANE_CLOSURE.csv").drop(columns="Unnamed: 0")
DF1

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"['River Av', 'End']","['Eastbound', 'Westbound']",August 16 2021,October 16 2021,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"['Carter Av', 'Weatherdon Av']",['Southbound'],June 14 2021,October 29 2021,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"['Pacific Av', 'Rupert Av']",['Southbound'],August 14 2020,October 16 2021,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"['Austin St', 'End']",['Westbound'],May 10 2021,November 30 2021,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"['Tache Av', 'St Joseph St']",['Eastbound'],May 31 2021,January 13 2022,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
425,Sherbrook,Logan,"['Logan Av', 'William Av']",['Southbound'],September 20 2021,November 01 2021,12:00 AM,12:00 AM,No,49.906584,-97.153705,"[[(49.904179261579, -97.155653416614), (49.904...",1,30
426,Gull Lake,Markham,"['Markham Rd', 'East Lake Dr']","['Northbound', 'Southbound']",September 17 2021,October 08 2021,12:00 AM,12:00 AM,No,49.809311,-97.158976,"[[(49.80730904568, -97.158914003228), (49.8081...",1,39
427,Wardlaw,Osborne,"['Osborne St', 'Scott St']",['Eastbound'],September 20 2021,October 15 2021,12:00 AM,12:00 AM,No,49.877534,-97.142232,"[[(49.876961712509, -97.143938755153), (49.877...",1,5
428,Scotsborough,Beckinsale,"['Beckinsale By', 'Novavista Dr']",['Southbound'],September 23 2021,October 13 2021,12:00 AM,12:00 AM,No,49.820905,-97.107388,"[[(49.820524519173, -97.107184511392), (49.820...",1,4


In [11]:
DF = pd.concat([DF1,DF])
DF

,Street,At,Boundaries,Direction,Date Closed - From,Date Closed - To,Time Closed - From,Time Closed - To,Complete Closure,Lat,Long,Geometry,Num_Segments,Num_Points
0,Bryce,River,"['River Av', 'End']","['Eastbound', 'Westbound']",August 16 2021,October 16 2021,12:00 AM,12:00 AM,No,49.879699,-97.142781,"[[(49.879153816388, -97.142387643857), (49.880...",1,3
1,Lilac,Carter,"['Carter Av', 'Weatherdon Av']",['Southbound'],June 14 2021,October 29 2021,12:00 AM,12:00 AM,No,49.862454,-97.151445,"[[(49.862119196838, -97.151173092203), (49.862...",1,5
2,Princess,Pacific,"['Pacific Av', 'Rupert Av']",['Southbound'],August 14 2020,October 16 2021,12:00 AM,12:00 AM,No,49.901786,-97.139766,"[[(49.90158994935, -97.139915854251), (49.9019...",1,3
3,Henry,Austin,"['Austin St', 'End']",['Westbound'],May 10 2021,November 30 2021,12:00 AM,11:00 PM,No,49.903340,-97.134152,"[[(49.903503694415, -97.13466284795), (49.9033...",1,4
4,Grandin,Tache,"['Tache Av', 'St Joseph St']",['Eastbound'],May 31 2021,January 13 2022,12:00 AM,12:00 AM,No,49.896013,-97.126315,"[[(49.895678857718, -97.127616387474), (49.896...",1,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
154,Kenaston,South Town,"[South Town Rd, Bison Dr]",[Northbound],2021-08-12 00:00:00,2022-04-30 00:00:00,12:00 AM,12:00 AM,No,49.793520,-97.193865,"[[(49.79534709946, -97.194058236482), (49.7941...",1,56
155,Smith,York,"[York Av, St Mary Av]",[Northbound],2019-07-08 00:00:00,2021-12-17 00:00:00,09:00 AM,02:00 PM,No,49.890207,-97.140148,"[[(49.889472952638, -97.139749173077), (49.889...",1,4
156,Juno,McDermot,"[McDermot Av, Notre Dame Av]","[Northbound, Southbound]",2021-02-21 00:00:00,2021-12-30 00:00:00,12:00 AM,12:00 AM,No,49.900115,-97.152503,"[[(49.899456360592, -97.153010204315), (49.899...",1,4
157,Fort,Graham,"[Graham Av, Portage Av]",[Northbound],2021-08-17 00:00:00,2023-10-27 00:00:00,12:00 AM,12:00 AM,No,49.893978,-97.139238,"[[(49.893005589417, -97.138726184172), (49.893...",1,6


In [ ]:
START_DATE = pd.to_datetime('Aug 1 2021'); END_DATE = pd.to_datetime('Sep 1 2021')
cond = (START_DATE > DF["Date Closed - From"]) & (DF["Date Closed - To"] >= END_DATE)
DF_AUG = DF[cond]
DF_AUG

In [ ]:
# DF.to_csv("clean_datasets/LANE_CLOSURE.csv")
DF_AUG.to_csv("clean_datasets/LANE_CLOSURE_AUG_2021.csv")

# **5.road_network**

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
DF = pd.read_csv("../../datasets/road_network.csv",low_memory=False)
for col in DF.columns:
    print(col,DF[col].dtype)
DF

ID int64
Block ID float64
Street Name object
Street Type object
Street Direction object
Street Class object
Street Type Secondary object
Street Qualifier object
Street Qualifier Secondary object
Address From Left float64
Address To Left float64
Address From Right float64
Address To Right float64
Number of Lanes float64
Oneway int64
Has Reversed Geometry float64
Speed Limit int64
Speed Limit Description object
Speed Limit Jurisdiction object
Location object


,ID,Block ID,Street Name,Street Type,Street Direction,Street Class,Street Type Secondary,Street Qualifier,Street Qualifier Secondary,Address From Left,Address To Left,Address From Right,Address To Right,Number of Lanes,Oneway,Has Reversed Geometry,Speed Limit,Speed Limit Description,Speed Limit Jurisdiction,Location
0,23480,1873.0,Sifton,Rd,NaN,Arterial,Street,A,NaN,NaN,NaN,NaN,NaN,2.0,0,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.143794801994 49.81055738...
1,6648,23974.0,Valde,Ave,NaN,Local,Street,A,NaN,19.0,19.0,22.0,50.0,2.0,0,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.003620088172 49.91094071...
2,15347,23779.0,Lagimodiere,Blvd,NaN,Arterial,Street,A,NaN,NaN,NaN,NaN,NaN,2.0,1,0.0,80,Posted 80 kph speed limit,City,MULTILINESTRING ((-97.066599484474 49.86715578...
3,30035,70342.0,Sage Creek,Blvd,NaN,Collector,Street,B,NaN,NaN,NaN,0.0,0.0,2.0,2,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.049568380871 49.83374749...
4,19001,24031.0,Point,Rd,NaN,Collector,Street,A,NaN,NaN,NaN,NaN,NaN,2.0,0,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.145356293446 49.84697663...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28957,29646,17669.0,Newton,Ave,NaN,Local,Street,A,NaN,258.0,328.0,255.0,325.0,2.0,0,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.11556803066 49.943945001...
28958,8062,23040.0,Main,St,NaN,Arterial,Street,A,NaN,782.0,782.0,NaN,NaN,3.0,1,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.134404708765 49.90567402...
28959,6592,18560.0,Digby,Ave,NaN,Local,Street,A,NaN,15.0,27.0,NaN,NaN,2.0,0,0.0,50,Speed Limits are less than or equal to 50 kph,City_Transportation,MULTILINESTRING ((-97.125132844509 49.91383316...
28960,8584,24835.0,Disraeli,Fwy,NaN,Arterial,Street,B,NaN,NaN,NaN,NaN,NaN,3.0,2,0.0,60,Posted 60 kph speed limit,City,MULTILINESTRING ((-97.128799417643 49.90276360...


In [3]:
DF = DF.loc[:,["Block ID","Street Name","Street Type","Number of Lanes","Oneway","Has Reversed Geometry","Speed Limit","Location"]].rename(columns={"Street Name":"Street","Has Reversed Geometry":"Reversed"})
__clean_string = lambda l: np.array([[(float(j.split()[1]),float(j.split()[0])) for j in m] for m in l],dtype=object)
DF.loc[:,"Location"] = __clean_string(DF["Location"].str.replace("MULTILINESTRING \(\(|\)\)","",regex=True).str.split(", "))
DF.loc[:,"Num_Points"] = [len(i) for i in DF["Location"].values]
DF

,Block ID,Street,Street Type,Number of Lanes,Oneway,Reversed,Speed Limit,Location,Num_Points
0,1873.0,Sifton,Rd,2.0,0,0.0,50,"[(49.810557383041, -97.143794801994), (49.8105...",2
1,23974.0,Valde,Ave,2.0,0,0.0,50,"[(49.910940712709, -97.003620088172), (49.9109...",2
2,23779.0,Lagimodiere,Blvd,2.0,1,0.0,80,"[(49.8671557845, -97.066599484474), (49.869726...",2
3,70342.0,Sage Creek,Blvd,2.0,2,0.0,50,"[(49.833747499855, -97.049568380871), (49.8337...",2
4,24031.0,Point,Rd,2.0,0,0.0,50,"[(49.846976634414, -97.145356293446), (49.8472...",2
...,...,...,...,...,...,...,...,...,...
28957,17669.0,Newton,Ave,2.0,0,0.0,50,"[(49.943945001092, -97.11556803066), (49.94406...",21
28958,23040.0,Main,St,3.0,1,0.0,50,"[(49.905674025615, -97.134404708765), (49.9062...",5
28959,18560.0,Digby,Ave,2.0,0,0.0,50,"[(49.913833165245, -97.125132844509), (49.9138...",34
28960,24835.0,Disraeli,Fwy,3.0,2,0.0,60,"[(49.902763600349, -97.128799417643), (49.9028...",14


In [ ]:
DF.to_csv("clean_datasets/ROAD_NETWORK.csv")

In [ ]:
# [" ".join(map(str,val)) for val in DF[["Street Name","Street Type","Street Direction"]].fillna("").values]